# 釣果コメントの分析サンプル

In [1]:
from janome.tokenizer import Tokenizer
import pandas as pd

In [2]:
import sqlite3
conn = sqlite3.connect("../data/fishing_result.db")
sql = '''
select * from fishing_comments c limit 3
'''
comments=pd.read_sql_query(sql, conn)
comments

,index,Date,Point,Weather,WaterTemp,Quantity,Comment,Tide,Time
0,0,2017-12-30,daikoku,晴,12.0,150.0,本日は2017年の最終営業日でした。釣果が厳しい中、多くのお客様にお越しいただきまして...,中潮,None
1,0,2017-12-29,daikoku,晴,12.8,132.0,本日はご来場頂きありがとうございます。今日は風も弱く、波穏やかな冬晴れの一日となりま...,中潮,2017-12-29 17:25:00
2,0,2017-12-28,daikoku,晴,12.9,58.0,本日はご来場頂き誠にありがとうございます。今朝は厳しい寒さでしたが日中は風も無く...,若潮,2017-12-28 17:28:00


In [3]:
t = Tokenizer()
words=pd.DataFrame()

for comment in comments.to_dict('records'):
    # コメントを形態素解析
    print(comment['Comment'])
    if not comment['Comment']:
        continue
    tokens = t.tokenize(comment['Comment'])
    #print(len(list(tokens)))
    #if not tokens:
    #    continue
    df_token=pd.DataFrame()
    for token in tokens:
        str_token=str(token).replace("\t",",")
        token_list=str_token.split(",")
        #print(token_list)
        df_token = df_token.append(pd.Series(token_list),ignore_index=True)
    print(df_token.size)
    df_token = df_token.rename(columns={0:"単語",1:"品詞",2:"品詞細分類1"})

    #名詞（代名詞、非自立名詞をのぞく）を抽出
    noun = df_token[df_token["品詞"]=="名詞"]
    noun2=noun[(noun["品詞細分類1"] != "代名詞") & (noun["品詞細分類1"] != "非自立")]
    noun3 = pd.DataFrame(noun2["単語"])
    noun3['Date'] = comment['Date']
    noun3['Point'] = comment['Point']
    words = words.append(noun3, ignore_index=True)

本日は2017年の最終営業日でした。釣果が厳しい中、多くのお客様にお越しいただきまして有り難うございました。年明けは1月2日(月)6:00から通常営業いたします。(釣り餌売店の営業は7:00からです。ご注意ください。)本年も多くのお客様にご愛顧頂きまして、本当に有り難うございました。来年も安全で楽しくお過ごし頂ける施設運営に従業員一同励んでまいります。2018年も皆様にとって良い年となる事を祈願いたします。(17:30、大黒海づり施設従業員一同)
1400
本日はご来場頂きありがとうございます。今日は風も弱く、波穏やかな冬晴れの一日となりました。釣果は厳しく、朝方にアジ、日中にはウミタナゴが数を伸ばしました。夕方には、アジは来ましたが場所が限られたうえ一時で終わってしまいました。明日は今年の最終日になります。多くの皆様のご来場をお待ちしております。※12/1〜12/30は、時間延長イベントとして「6:00」から営業します。ただし釣り餌売店は「7:00」からとなりますので、エサや仕掛けなどの準備にご注意ください。※「釣果情報」につきましては、お帰り時にお聞きした集計となります。(大黒17:25)
1920
本日はご来場頂き誠にありがとうございます。今朝は厳しい寒さでしたが日中は風も無く釣り日和となりました。釣果は気温の低さと同様厳しくウミタナゴが数を伸ばしてくれましたアジは3時30分頃より桟橋内側先端にて回遊があり数は伸びなやみました。明日も晴れて気温低めです、暖かな服装でお越しください。尚、明日は『濱ともカード』の日です、受付で提示されますと料金半額です。(大黒17:28)※釣果情報は、お帰り時の聞き取りの集計となります。イベントとして6:00〜17:00で営業しています。売店は7:00〜16:00となりますので、エサや仕掛けなどの準備にご注意ください。
1860


In [4]:
words.describe()

,単語,Date,Point
count,252,252,252
unique,116,3,1
top,゙,2017-12-28,daikoku
freq,20,93,252


In [5]:
wc = words.groupby(['単語']).size()

In [6]:
wc2 = wc.reset_index(name='count').sort_values(['count'], ascending=True)
wc2.head()

,単語,count
57,内側,1
74,年明け,1
71,安全,1
68,夕方,1
66,場所,1
